In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkSession.builder.appName("Linreg").getOrCreate()

24/10/30 20:11:24 WARN Utils: Your hostname, prateek-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.86.4.180 instead (on interface enp0s3)
24/10/30 20:11:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/30 20:11:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/30 20:11:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/30 20:11:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = sc.read.csv("Files/test1.csv",header=True,inferSchema=True)
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [8]:
import numpy as np
from pyspark.ml.feature import VectorAssembler

In [15]:
features = VectorAssembler(inputCols=['age','Experience'],outputCol='Independent features')

In [16]:
output = features.transform(df)

In [17]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [18]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent features']

In [19]:
final_df = output.select("Independent features","Salary")
final_df.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [20]:
from pyspark.ml.regression import LinearRegression
train_data , test_data = final_df.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="Independent features",labelCol="Salary")
regressor = regressor.fit(train_data)

24/10/30 20:18:22 WARN Instrumentation: [af188e7d] regParam is zero, which might cause numerical instability and overfitting.
24/10/30 20:18:23 WARN Instrumentation: [af188e7d] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [21]:
regressor

LinearRegressionModel: uid=LinearRegression_f5486cc5c1e8, numFeatures=2

In [22]:
regressor.coefficients

DenseVector([750.0, 1500.0])

In [23]:
regressor.intercept

-2249.9999999995152

In [24]:
pred = regressor.evaluate(test_data)

In [27]:
pred.predictions.show()

+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|20249.999999999938|
|          [29.0,4.0]| 20000| 25499.99999999982|
|          [30.0,8.0]| 25000|32249.999999999756|
|         [31.0,10.0]| 30000| 35999.99999999971|
+--------------------+------+------------------+



In [28]:
pred.meanAbsoluteError, pred.meanSquaredError

(4749.999999999806, 29718749.99999775)